In [10]:
# импортируем необходимые библиотеки
!pip install pympi-ling datasets wandb accelerate evaluate jiwer gradio -qU

In [11]:
# предоставляем доступ к файлам, хранящимся в аккаунте Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
from collections import Counter
from dataclasses import dataclass
from evaluate import load
from pathlib import Path
from transformers import Trainer, TrainingArguments, Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC
from transformers import Wav2Vec2Processor

import datasets
import json
import numpy as np
import pandas as pd
import pympi
import torch
import torchaudio
import wandb

In [13]:
# сохраняем полный путь до папки, где хранятся данные
data_folder = '/content/drive/MyDrive/Linguistics'
# используемая модель - facebook/wav2vec2-large-xlsr-53
model_name = 'facebook/wav2vec2-large-xlsr-53'

# сохраняем полный путь до папки, где хранятся файлы в формате .wav
wav_folder = f"{data_folder}/wav/"
# сохраняем полный путь до папки, где хранятся файлы в формате .eaf
eaf_folder = f"{data_folder}/eaf/"

In [15]:
# будем сохранять данные в таком формате:
# path - полный путь до wav-файла
# start - начало отрезка речи в миллисекундах от начала записи
# end - конец отрезка речи в миллисекундах от начала записи
# text - расшифровка речи на заданном отрезке
csv_data = {"path": [], "start": [], "end": [], "text": []}

for wav_path in Path(wav_folder).iterdir():
  # обрабатываем сценарий, когда очередной файл имеет расширение не .wav
  if not wav_path.name.endswith('.wav'):
    continue
  # сопоставляем файлу .wav файл .eaf, имеющий то же название с
  # добавлением "_transliterated" в конце
  eaf_path = Path(eaf_folder, wav_path.name).with_suffix('.eaf')
  new_file_name = eaf_path.stem + '_transliterated.eaf'
  eaf_path = Path(eaf_folder, new_file_name)
  # обрабатываем сценарий, когда необходимого файла не оказывается
  if not eaf_path.exists():
    print(f"No EAF file is found for WAV: {wav_path.name}")
    continue

  starts = []
  ends = []
  texts = []
  paths = []
  speech_array, sampling_rate = torchaudio.load(wav_path)
  transcriptions = \
    pympi.Elan.Eaf(eaf_path).get_annotation_data_for_tier('sentFon')

  # сохраняем корректные данные для добавления в датасет, описанный выше
  for transcription in transcriptions:
    start, end, text = transcription
    start_sample = int(start / 1000 * sampling_rate)
    end_sample = int(end / 1000 * sampling_rate)
    # обрабатываем сценарий, когда временные рамки в расшифровке выходят
    # за пределы длины аудиозаписи
    if start_sample >= speech_array.shape[1] or \
        end_sample > speech_array.shape[1]:
      print(f"Invalid start or end sample indices for file {wav_path.name}: \
              start={start_sample}, end={end_sample}, \
              length={speech_array.shape[1]}")
      starts = []
      ends = []
      texts = []
      paths = []
      break
    starts.append(start)
    ends.append(end)
    texts.append(text)
    paths.append('wav/' + wav_path.name)
  csv_data["start"].extend(starts)
  csv_data["end"].extend(ends)
  csv_data["text"].extend(texts)
  csv_data["path"].extend(paths)

Invalid start or end sample indices for file 2014_Uchami_Mukto_Stalina_LB1.wav:               start=11988320, end=12230208,               length=12226560
Invalid start or end sample indices for file 2011_Hantayskoye_Ozero_Chemprogir_Antonina_Dmitriyevna_LB.wav:               start=12248334, end=12361406,               length=12357120
Invalid start or end sample indices for file 2009_Sym_Boyarin_Georgiy_LR3.wav:               start=8730918, end=8871156,               length=8774784
Invalid start or end sample indices for file 2008_Tutonchany_Uvachan_Inna_LAv3.wav:               start=45980335, end=46447884,               length=46444192
Invalid start or end sample indices for file 2008_Tutonchany_Lapushkina_LA2.wav:               start=7495985, end=7888872,               length=7684608
Invalid start or end sample indices for file 2008_Tutonchany_Khukochar_Danil_LR.wav:               start=12923040, end=13656000,               length=13653120
Invalid start or end sample indices for file 

In [16]:
#для токенизации посчитаем число упоминаний каждого символа в расшифровках
all_chars_counts = Counter([char for text in csv_data['text'] \
                            for char in list(text)])

# будем игнорировать пунктуационные знаки и специальные символы
chars_to_ignore = ',?.!-;:"“%‘”�{}<>*()=[]'

# создаём словарь символов, опуская игнорируемые символы, а также символы,
# встречающиеся реже 10 раз
chars_vocab = {}
for i, (k, v) in enumerate(all_chars_counts.most_common()):
  if k in chars_to_ignore:
    continue
  if v < 10:
    continue
  chars_vocab[k] = i

for i, k in enumerate(chars_vocab.keys()):
  chars_vocab[k] = i

# преобразуем данные согласно описанному подходу к токенизации
idx_to_del = []
for i in range(len(csv_data['text'])):
  text = ''.join(char for char in csv_data['text'][i] \
                 if char in chars_vocab.keys())
  if text:
    csv_data['text'][i] = text
  else:
    idx_to_del.append(i)

for col in ["path", "start", "end", "text"]:
  csv_data[col] = [el for i, el in enumerate(csv_data[col]) \
                   if i not in idx_to_del]

chars_vocab["|"] = chars_vocab[" "]
del chars_vocab[" "]

# добавляем специальные токены UNK и PAD
chars_vocab["[UNK]"] = len(chars_vocab)
chars_vocab["[PAD]"] = len(chars_vocab)

# сохраняем словарь для дальнейшего использования
with open('vocab.json', 'w') as vocab_file:
    json.dump(chars_vocab, vocab_file)

len(chars_vocab)

80

In [17]:
# загружаем токенизатор
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json",
                                 unk_token="[UNK]",
                                 pad_token="[PAD]",
                                 word_delimiter_token="|")

# загружаем "извлекатель признаков"
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=True)

# создаём процессор, состоящий из токенизатора и "извлекателя признаков"
processor = Wav2Vec2Processor(feature_extractor=feature_extractor,
                              tokenizer=tokenizer)

In [18]:
# создаём датасет
df = pd.DataFrame(csv_data).sample(frac=1.0, random_state=42)
sample_size = df.shape[0]

# обрабатываем сценарий, когда начало речи отмечено позже, чем её конец
df = df[df['start'] < df['end']]
sample_size = df.shape[0]

df["path"] = df["path"].apply(lambda x: data_folder + "/" + x)
train_num = int(df.shape[0] * 0.7)
val_num = int(df.shape[0] * 0.2)

# делим выборка на обучающую, валидационную и тестовую в соотношении 70:20:10
df.iloc[:train_num].to_csv('train.csv', index=False)
df.iloc[train_num:val_num + train_num].to_csv('val.csv', index=False)
df.iloc[val_num + train_num:].to_csv('test.csv', index=False)
df.head()

,path,start,end,text
3688,/content/drive/MyDrive/Linguistics/wav/2007_St...,242130,243107,jūllən
5444,/content/drive/MyDrive/Linguistics/wav/2007_Ch...,170762,179982,ə taduk urīďanəl hərgīgit urīďanəl tawər ďəwďə...
4477,/content/drive/MyDrive/Linguistics/wav/2007_Ek...,54488,58735,oldomośiliŋnərəw oldolwo turumďənəl wojnaŋəhi
4342,/content/drive/MyDrive/Linguistics/wav/2007_Ek...,196420,198395,əhikon nuŋan ińd’ərən uš’amidū
3889,/content/drive/MyDrive/Linguistics/wav/2007_St...,839618,849949,amargūt amargūt nəlki bi tarə irgičilnun gēwuØ...


In [19]:
# сохраняем данные в соответствующие файлы
data_files = {
    "train": "train.csv",
    "val": "val.csv",
    "test": "test.csv",
}

dataset = datasets.load_dataset("csv", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [20]:
def prepare_dataset(batch):
  """ Данная функция подготавливает датасет к работе над ним """

  speech_array, sampling_rate = torchaudio.load(batch["path"])
  # обрабатываем сценарий, когда файл записан в стерео формате, и превращаем
  # его в моно формат путём усреднения данных
  if speech_array.shape[0] == 2:
    speech_array = torch.mean(speech_array, dim=0, keepdim=True)

  start_sample = int(batch["start"] / 1000 * sampling_rate)
  end_sample = int(batch["end"] / 1000 * sampling_rate) + 1

  # обрабатываем сценарий, когда временные рамки в расшифровке выходят
  # за пределы длины аудиозаписи
  if start_sample >= speech_array.shape[1] or end_sample > speech_array.shape[1]:
    raise ValueError(f"Invalid start or end sample indices \
                      for file {batch['path']}: start={start_sample}, \
                      end={end_sample}, length={speech_array.shape[1]}")

  # обрабатываем сценарий, когда отрезок речи оказывается пустым
  speech_segment = speech_array[:, start_sample:end_sample]
  if speech_segment.shape[1] == 0:
    raise ValueError(f"Extracted segment is empty for file {batch['path']}: \
                      start={start_sample}, end={end_sample}")

  # битрейт 16000 должен быть у всех файлов
  resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
  speech_segment = resampler(speech_segment)

  batch["input_values"] = processor(speech_segment.squeeze(), sampling_rate=16000).input_values[0]

  with processor.as_target_processor():
    batch["labels"] = processor(batch["text"]).input_ids

  return batch

In [21]:
# применяем функцию prepare_dataset ко всем выборкам
dataset = dataset.map(prepare_dataset, \
                      remove_columns=["path", "start", "end", "text"], \
                      num_proc=2)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/5096 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map (num_proc=2):   0%|          | 0/1456 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map (num_proc=2):   0%|          | 0/729 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [22]:
# необходимо вставить уникальный токен WANDB для построения графиков
# обучения на сайте https://wandb.ai/
WANDB_TOKEN = ""

In [23]:
# создаём "упаковщика" данных, превращающего данные в батчи

@dataclass
class DataCollator:

  processor: Wav2Vec2Processor
  max_length: int | None = 200_000

  def __call__(self, features):
    input_features = [{"input_values": f["input_values"]} for f in features]
    label_features = [{"input_ids": f["labels"]} for f in features]

    batch = self.processor.pad(
      input_features,
      padding=True,
      max_length=self.max_length,
      return_tensors="pt",
    )

    with self.processor.as_target_processor():
      labels_batch = self.processor.pad(
        label_features,
        padding=True,
        max_length=self.max_length,
        return_tensors="pt",
      )

    labels = \
      labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1),
                                            -100)
    batch["labels"] = labels
    return batch

data_collator = DataCollator(processor=processor)

In [24]:
# реализуем подсчёт метрики WER
wer_metric = load("wer")

def compute_metrics(pred):
  """ Данная функция возвращает значение метрики WER
      для очередного предсказания """

  pred_ids = np.argmax(pred.predictions, axis=-1)
  pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

  pred_str = processor.batch_decode(pred_ids)
  label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

  wer = wer_metric.compute(predictions=pred_str, references=label_str) * 100

  return {'wer': wer}

In [25]:
# финальная версия модели
model = Wav2Vec2ForCTC.from_pretrained(
    model_name,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    vocab_size=len(processor.tokenizer)
)
model.freeze_feature_extractor()
model.gradient_checkpointing_enable()

training_args = TrainingArguments(
  output_dir="./results",
  group_by_length=True,
  eval_strategy="steps",
  per_device_train_batch_size=4,
  gradient_accumulation_steps=8,
  learning_rate=3e-4,
  weight_decay=0.005,
  warmup_steps=500,
  save_steps=500,
  eval_steps=100,
  save_total_limit=2,
  logging_steps=50,
  num_train_epochs=15,
  report_to="wandb",
  fp16=True,
  gradient_checkpointing=True,
  load_best_model_at_end=True,
  metric_for_best_model="wer",
  greater_is_better=False,
)


trainer = Trainer(
  model=model,
  args=training_args,
  data_collator=data_collator,
  train_dataset=dataset["train"],
  eval_dataset=dataset["val"],
  compute_metrics=compute_metrics,
  tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transfo

In [26]:
# авторизуемся при помощи ключа на сайте https://wandb.ai/
wandb.login(key=WANDB_TOKEN)

wandb: Currently logged in as: ponomarchuk_anna. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [27]:
# запускаем обучение, передаём данные на сайт https://wandb.ai/
run = wandb.init()
trainer.train()
run.finish()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In 

Step,Training Loss,Validation Loss,Wer
100,8.862500,4.767185,100.000000
200,3.593300,3.512512,100.000000
300,3.494200,3.485154,100.000000
400,3.505300,3.488468,100.000000
500,3.218500,2.887340,100.000000
600,2.207600,1.623664,99.984353
700,1.612900,1.275407,92.583320
800,1.599500,1.158489,84.603348
900,1.333000,1.068937,80.488187
1000,1.257100,1.028337,77.483962


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/di

eval/loss,█▆▆▆▅▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,█▅▄▇▄▁▂▃▆▅▂▂▆▅▅▃█▅▅▇▆▇▇
eval/samples_per_second,▁▄▅▂▅█▇▆▃▃▇▇▃▄▄▅▁▄▄▂▃▂▂
eval/steps_per_second,▁▄▅▂▅█▇▆▃▄▇▇▃▄▄▅▁▄▅▂▃▂▂
eval/wer,██████▆▅▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,█▅▁▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▃▁▁▁▁▁▁▂▂▁▁▁▁▁
train/learning_rate,▂▂▃▄▄▅▇▇███▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▁▁
train/loss,█▄▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,0.87283


In [28]:
def map_to_result(batch):
  """ Данная функция возвращает предсказание модели для батча данных """

  with torch.no_grad():
    input_values = \
      torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)

  return batch

In [29]:
# получаем предсказания для тестовой выборки
results = dataset["test"].map(map_to_result, batch_size=8)

Parameter 'function'=<function map_to_result at 0x7ab5ad1f8430> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/729 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [30]:
# оцениваем полученные предсказания при помощи выбранной метрики
print("Test WER: {:.3f}".format(wer_metric.compute(
    predictions=results["pred_str"], references=results["text"]) * 100))

Test WER: 77.902


In [31]:
# входим в аккаунт на huggingface_hub через уникальный токен

from huggingface_hub import notebook_login
notebook_login()

In [32]:
# пушим полученную модель на huggingface_hub для создания gradio-приложения

kwargs = {
    "dataset": "Evenki",
    "language": "ev",
    "model_name": "wav2vec2-large-xlsr-53",
}

trainer.push_to_hub(**kwargs)

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/annaponomarchuk/results/commit/7134bd3f7a3ef6d512d3ea24a16b58e7f58c1a96', commit_message='End of training', commit_description='', oid='7134bd3f7a3ef6d512d3ea24a16b58e7f58c1a96', pr_url=None, pr_revision=None, pr_num=None)

In [33]:
import gradio as gr
from transformers import pipeline

In [34]:
processor.save_pretrained(training_args.output_dir)

[]

In [35]:
pipe = pipeline(model='annaponomarchuk/results')

def transcribe(audio):
  """ Данная функция преобразует переданный аудиофайл в текст """
  text = pipe(audio)['text']
  return text

interface = gr.Interface(
  fn=transcribe,
  inputs=gr.Audio(sources='upload', type='filepath'),
  outputs='text',
  title='Evenki Speech-to-Text',
)

interface.launch()

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fa9906e78d2845537b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
